`직접 transform 안하고 내부적으로 transform 진행하는 skicit-learn으로 진행하는 방법으로 구현`

In [1]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error

def evaluate_reg_all(y_test, y_predict):
    MSE = mean_squared_error(y_test,y_predict,squared=True)
    RMSE = mean_squared_error(y_test,y_predict,squared=False)
    MAE = mean_absolute_error(y_test,y_predict)
    R2 = r2_score(y_test,y_predict)
    
    print(f'MSE: {MSE:.3f}, RMSE: {RMSE:.3F}, MAE: {MAE:.3F}, R^2: {R2:.3F}')

In [6]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import numpy as np

# 학습 단계
## 데이터 로드

train_df = pd.read_csv('./data/train_pre.csv')
test_df = pd.read_csv('./data/test.csv')

## 데이터 분석

## 데이터 전처리

x_train = train_df.drop(['가격'], axis=1)
x_test = test_df.drop(['가격'], axis=1)
y_train = train_df['가격']
y_test = test_df['가격']

# print(x_train.head())
'''
     년식  종류    연비   마력    토크   연료  하이브리드   배기량    중량 변속기
0  2015  대형   6.8  159  23.0  LPG      0  2359  1935  수동
1  2012  소형  13.3  108  13.9  가솔린      0  1396  1035  자동
2  2015  중형  14.4  184  41.0   디젤      0  1995  1792  자동
3  2015  대형  10.9  175  46.0   디젤      0  2497  2210  수동
4  2015  대형   6.4  159  23.0  LPG      0  2359  1935  자동
'''
# print(x_train.columns)
'''Index(['년식', '종류', '연비', '마력', '토크', '연료', '하이브리드', '배기량', '중량', '변속기'], dtype='object')'''


transformer = make_column_transformer(
    (OneHotEncoder(), ['종류', '연료', '변속기']),
    remainder='passthrough')

## 모델 생성

#파이프라인을 이용하여 전처리와 모델 순차적으로 실행되도록 함
model = make_pipeline(transformer,LinearRegression())  # 데이터가 transform 먼저한 후에 model에 들어옴

## 모델 학습

model.fit(x_train, y_train)

## 모델 검증

print(f'훈련_R2: {model.score(x_train, y_train)}') 
print(f'테스트_R2: {model.score(x_test, y_test)}') 
print('----------------------------------------------------------------')
## 모델 평가 (회귀모델 평가지표 출력)
y_predict = model.predict(x_test) ## 주의: 테스트 데이터에 대해 평가해야함!
evaluate_reg_all(y_test,y_predict)
print('----------------------------------------------------------------')

# 예측 단계(모델 예측)

## 신규 데이터 수집
x_real = np.array([
    [2015, '대형', 6.8, 159, 23, 'LPG', 0,2359, 1935, '수동']
])

x_real_df = pd.DataFrame(x_real, columns=['년식', '종류', '연비', '마력', '토크', '연료', '하이브리드', '배기량', '중량', '변속기'])

# 인코딩 및 결과 -> 인코딩이 model 안에 들어가 있어서 transform 필요없음.
y_predict = model.predict(x_real_df)  # model안에 trainsform까지 포함된 모델임. 따라서 transform 호출할 필요 없음.
print(f'예측값: {y_predict[0]}')


훈련_R2: 0.8849618214729724
테스트_R2: 0.7746849250515475
----------------------------------------------------------------
MSE: 1397856.930, RMSE: 1182.310, MAE: 570.120, R^2: 0.775
----------------------------------------------------------------
예측값: 1796.2824454478687
